In [2]:
class Persona:
    def __init__(self, nombre):
        self.nombre = nombre

class Cliente(Persona):
    def __init__(self, nombre):
        super().__init__(nombre)
        self.historial_pedidos = []
        self.puntos_lealtad = 0

    def realizar_pedido(self, pedido, inventario, promocion=None):
        if pedido.verificar_disponibilidad(inventario):
            if promocion and self.puntos_lealtad >= 50:
                promocion.aplicar_descuento(pedido)
            self.historial_pedidos.append(pedido)
            self.puntos_lealtad += 10
            pedido.actualizar_stock(inventario)
            pedido.actualizar_estado("En preparación")
            print("Productos:")
            for producto in pedido.productos:
                print(f"- {producto.nombre}: ${producto.precio}")
            print(f"Total con descuento: ${pedido.calcular_total()}")
        else:
            print("El pedido no pude ser realizado ya que no hay los insumos suficientes")

    def consultar_historial(self):
        print(f"Historial pedidos de {self.nombre}")
        for pedido in self.historial_pedidos:
            print(f"Pedido con total: ${pedido.calcular_total()} - Estado: {pedido.estado}")

class Empleado(Persona):
    def __init__(self, nombre, rol):
        super().__init__(nombre)
        self.rol = rol

    def actualizar_inventario(self, inventario, ingrediente, cantidad):
        inventario.agregar_ingrediente(ingrediente, cantidad)       
        print(f"Se han añadido {cantidad} unidades de {ingrediente}") 

class ProductoBase:
    def __init__(self, nombre, precio):
        self.nombre = nombre
        self.precio = precio
        self.ingredientes = {}

class Bebida(ProductoBase):
    def __init__(self, nombre, precio, tamaño, tipo, personalizacion={}):
        super().__init__(nombre, precio)
        self.tamaño = tamaño
        self.tipo = tipo
        self.personalizacion = personalizacion
        self.ingredientes = {"Agua": 1}
        self.ingredientes.update(personalizacion)

    def descripcion(self):
        print(f" ☆ Nombre: {self.nombre}")
        print(f" ☆ Precio: ${self.precio}")
        print(f" ☆ Tamaño: {self.tamaño}")
        print("Ingredientes:")
        for ingrediente in self.ingredientes:
            print(f"- {ingrediente}")

class Postre(ProductoBase):
    def __init__(self, nombre, precio, personalizacion={}, vegano=False, sin_gluten=False):
        super().__init__(nombre, precio)
        self.vegano = vegano
        self.sin_gluten = sin_gluten
        self.personalizacion = personalizacion
        self.ingredientes = {"Harina": 1}
        self.ingredientes.update(personalizacion)

    def descripcion(self):
        print(f" ☆ Nombre: {self.nombre}")
        print(f" ☆ Precio: ${self.precio}")
        print(f" ☆ Vegano: {'Si' if self.vegano else 'No'}")
        print(f" ☆ Sin gluten: {'Si' if self.sin_gluten else 'No'}")
        print("Ingredientes:")
        for ingrediente in self.ingredientes:
            print(f"- {ingrediente}")

class Inventario:
    def __init__(self):
        self.stock = {}
    
    def agregar_articulo(self, ingrediente, cantidad):
        self.stock[ingrediente] = self.stock.get(ingrediente, 0) + cantidad

    def verificar_disponibilidad(self, ingredientes):
        faltantes = [ing for ing, cantidad in ingredientes.items() if self.stock.get(ing, 0) < cantidad]
        if faltantes:
            print(f" ☆ Faltan ingredientes: {', '.join(faltantes)}")
            return False
        return True

    def actualizar_stock(self, ingredientes):
        for ing, cantidad in ingredientes.items():
            if ing in self.stock:
                self.stock[ing] -= cantidad

    def consultar_inventario(self):
        print("Inventario actual:")
        for ing, cantidad in self.stock.items():
            print(f"☆ {ing}: {cantidad} unidades")


class Pedido:
    def __init__(self, cliente):
        self.cliente = cliente
        self.productos = []
        self.estado = "Pendiente"
        self.total = 0
        print(f" ☆ Se ha añadido un nuevo pedido para {self.cliente.nombre}")


    def agregar_producto(self, producto):
        self.productos.append(producto)
        self.total += producto.precio
        print(f" ☆ Se ha añadido {producto.nombre} con costo de ${producto.precio} al pedido de {self.cliente.nombre}")

    def calcular_total(self):
        return self.total

    def actualizar_estado(self, nuevo_estado):
        self.estado = nuevo_estado
        print(f" ☆ Nuevo estado del pedido: {self.estado}")

    def verificar_disponibilidad(self, inventario):
        return all(inventario.verificar_disponibilidad(prod.ingredientes) for prod in self.productos)

    def actualizar_stock(self, inventario):
        for prod in self.productos:
            inventario.actualizar_stock(prod.ingredientes)

class Promocion:
    def __init__(self, descripcion, descuento):
        self.descripcion = descripcion
        self.descuento = descuento
    
    def aplicar_descuento(self, pedido):
        pedido.total *= (1 - self.descuento / 100)

In [3]:
inventario = Inventario()
inventario.agregar_articulo("Café", 20)
inventario.agregar_articulo("Agua", 50)
inventario.agregar_articulo("Leche", 30)
inventario.agregar_articulo("Crema batida", 15)
inventario.agregar_articulo("Cheescake", 10)


In [4]:
inventario.consultar_inventario()

Inventario actual:
☆ Café: 20 unidades
☆ Agua: 50 unidades
☆ Leche: 30 unidades
☆ Crema batida: 15 unidades
☆ Cheescake: 10 unidades


In [5]:
cliente1 = Cliente("Feeeer")
cliente2 = Cliente("Pauuu")
cliente3 = Cliente("Emi")
empleado1 = Empleado("Vale", "Barista")
empleado2 = Empleado("Roberto", "Mesero")
empleado3 = Empleado("Anita", "Gerente")

In [6]:
bebida1 = Bebida("Café Moka Capuchino", 60, "Grande", "Caliente", {"Café": 1, "Leche deslactosada": 1, "Azúcar": 1, "Chocolate": 1})
bebida2 = Bebida("Capuccino Vegano", 50, "Mediano", "Caliente", {"Café": 1, "Leche de almendra": 1, "Azúcar": 1})
bebida3 = Bebida("Café con Leche", 50, "Mediano", "Caliente", {"Café": 1, "Leche entera": 1, "Azúcar": 1})
postre1 = Postre("Afogatto de Vainilla", 75, personalizacion={"Azúcar": 1, "Vainilla": 1})
postre2 = Postre("Galleta de avena", 30, personalizacion={"Azúcar": 1, "Avena": 1}, vegano=True)
postre3 = Postre("Brownie Sin Gluten", 30, personalizacion={"Azúcar": 1}, sin_gluten=True)

In [7]:
bebida1.descripcion()
postre1.descripcion()
bebida2.descripcion()
postre2.descripcion()
bebida3.descripcion()
postre3.descripcion()

 ☆ Nombre: Café Moka Capuchino
 ☆ Precio: $60
 ☆ Tamaño: Grande
Ingredientes:
- Agua
- Café
- Leche deslactosada
- Azúcar
- Chocolate
 ☆ Nombre: Afogatto de Vainilla
 ☆ Precio: $75
 ☆ Vegano: No
 ☆ Sin gluten: No
Ingredientes:
- Harina
- Azúcar
- Vainilla
 ☆ Nombre: Capuccino Vegano
 ☆ Precio: $50
 ☆ Tamaño: Mediano
Ingredientes:
- Agua
- Café
- Leche de almendra
- Azúcar
 ☆ Nombre: Galleta de avena
 ☆ Precio: $30
 ☆ Vegano: Si
 ☆ Sin gluten: No
Ingredientes:
- Harina
- Azúcar
- Avena
 ☆ Nombre: Café con Leche
 ☆ Precio: $50
 ☆ Tamaño: Mediano
Ingredientes:
- Agua
- Café
- Leche entera
- Azúcar
 ☆ Nombre: Brownie Sin Gluten
 ☆ Precio: $30
 ☆ Vegano: No
 ☆ Sin gluten: Si
Ingredientes:
- Harina
- Azúcar


In [8]:
pedido1 = Pedido(cliente1)
pedido1.agregar_producto(bebida1)
pedido1.agregar_producto(postre1)

pedido2 = Pedido(cliente2)
pedido2.agregar_producto(bebida2)
pedido2.agregar_producto(postre2)

pedido3 = Pedido(cliente3)
pedido3.agregar_producto(bebida3)
pedido3.agregar_producto(postre3)

 ☆ Se ha añadido un nuevo pedido para Feeeer
 ☆ Se ha añadido Café Moka Capuchino con costo de $60 al pedido de Feeeer
 ☆ Se ha añadido Afogatto de Vainilla con costo de $75 al pedido de Feeeer
 ☆ Se ha añadido un nuevo pedido para Pauuu
 ☆ Se ha añadido Capuccino Vegano con costo de $50 al pedido de Pauuu
 ☆ Se ha añadido Galleta de avena con costo de $30 al pedido de Pauuu
 ☆ Se ha añadido un nuevo pedido para Emi
 ☆ Se ha añadido Café con Leche con costo de $50 al pedido de Emi
 ☆ Se ha añadido Brownie Sin Gluten con costo de $30 al pedido de Emi


In [9]:
promo = Promocion("Descuento de 10% para clientes frecuentes", 10)

In [10]:
cliente1.realizar_pedido(pedido1, inventario, promo)
cliente2.realizar_pedido(pedido2, inventario, promo)
cliente3.realizar_pedido(pedido3, inventario, promo)

 ☆ Faltan ingredientes: Leche deslactosada, Azúcar, Chocolate
El pedido no pude ser realizado ya que no hay los insumos suficientes
 ☆ Faltan ingredientes: Leche de almendra, Azúcar
El pedido no pude ser realizado ya que no hay los insumos suficientes
 ☆ Faltan ingredientes: Leche entera, Azúcar
El pedido no pude ser realizado ya que no hay los insumos suficientes


In [11]:
pedido1.actualizar_estado("Entregado")
pedido2.actualizar_estado("Entregado")
pedido3.actualizar_estado("Entregado")

 ☆ Nuevo estado del pedido: Entregado
 ☆ Nuevo estado del pedido: Entregado
 ☆ Nuevo estado del pedido: Entregado


In [12]:
cliente1.consultar_historial()
cliente2.consultar_historial()
cliente3.consultar_historial()

Historial pedidos de Feeeer
Historial pedidos de Pauuu
Historial pedidos de Emi


In [13]:
inventario.consultar_inventario()

Inventario actual:
☆ Café: 20 unidades
☆ Agua: 50 unidades
☆ Leche: 30 unidades
☆ Crema batida: 15 unidades
☆ Cheescake: 10 unidades


In [ ]:
empleado1.actualizar_inventario(inventario, "Café", 5)

AttributeError: 'Inventario' object has no attribute 'agregar_ingrediente'